<a href="https://colab.research.google.com/github/Reennon/multigec-models/blob/main/notebooks/gemma_3_12b/multigec/omnigec_prediction_minimal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

from google.colab import userdata

os.environ["GIT_TOKEN"] = userdata.get('git_token')

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
!git clone https://$GIT_TOKEN@github.com/Reennon/omnigec-models.git

In [ ]:
%cd omnigec-models

In [ ]:
!git pull

In [ ]:
!pip install -U bitsandbytes peft accelerate datasets sentencepiece wandb python-dotenv wtpsplit -q
!pip install flash-attn --no-build-isolation -q
!pip install wtpsplit==2.1.1 -q
!pip install syntok==1.4.4 -q
!pip install omegaconf -q
!pip install wandb -q
!pip install --upgrade git+https://github.com/huggingface/transformers.git -q
!pip install --upgrade trl -q
!pip install pandas numpy -q

In [ ]:
import os

from omegaconf import OmegaConf
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from huggingface_hub import login
from src.utils.multigec import sentences, LANG_TO_CODE, LANG_CODE_TO_TOKEN
from src.utils.gemma_utils import inference_formatting_prompts_func as formatting_prompts_func
from langchain_core.prompts import PromptTemplate

from src.instruction_templates import multigec_prompts

import torch
import wandb

from transformers import BitsAndBytesConfig
from tqdm import tqdm
from trl.trainer import ConstantLengthDataset
import pandas as pd
from datasets import Dataset
from transformers.trainer_callback import EarlyStoppingCallback

from transformers import TrainingArguments
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig, PeftModelForCausalLM

tqdm.pandas()

In [ ]:
parameters = OmegaConf.load("./params/gemma_3_12b.yaml")

In [ ]:
track                   = "minimal"
model_name              = "gemma-3-12b-it"
fine_tuned_model_name   = f"gemma-3-12b-it-multigec-wikiedits-reddit-{track}"
experiment_name         = f"multigec-{track}-{model_name}"

hf_key                  = userdata.get("hf_key")
secret_wandb            = userdata.get("wandb_key")

in_path                 = f"/gdrive/MyDrive/omnigec/datasets/multigec_wikiedits_reddit_{track}.csv"
out_path                = f"/gdrive/MyDrive/omnigec/preds/multigec_wikiedits_reddit_test_{track}.csv"
out_model_dir           = f"/gdrive/MyDrive/omnigec/models/{fine_tuned_model_name}"

QUANTIZE_4BIT           = True
device                  = "cuda:0"

In [ ]:
login(hf_key)

In [ ]:
!env TORCH_USE_CUDA_DSA=1 -q

In [ ]:
base_model = "google/gemma-3-12b-it"
saved_checkpoint = out_model_dir + "/checkpoint-1800"

quantization_config = None
if QUANTIZE_4BIT:
  quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_use_double_quant=True,
      bnb_4bit_compute_dtype=torch.bfloat16,
      bnb_4bit_quant_storage=torch.bfloat16,
  )
tokenizer = AutoTokenizer.from_pretrained(saved_checkpoint)
config = AutoConfig.from_pretrained(base_model)
config.text_config.use_cache = False
base_model_instance = AutoModelForCausalLM.from_pretrained(
    base_model,
    config=config,
    quantization_config=quantization_config,
    torch_dtype=torch.bfloat16,
    device_map=device,
    attn_implementation="eager",
)
base_model_instance.resize_token_embeddings(len(tokenizer))
model = PeftModelForCausalLM.from_pretrained(
    base_model_instance,
    saved_checkpoint,
    torch_dtype=torch.bfloat16,
    device_map=device,
    ignore_mismatched_sizes=True
)


In [ ]:
test_df = pd.read_csv(out_path)

In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm

max_new_tokens = 1600
batch_size = 7
save_each = 7

# Assume test_df already exists and out_path is defined.
# Filter rows that need processing (target is empty/NaN).
to_process_df = test_df[test_df["target"].isna()].reset_index()  # preserve original index in "index" column

def collate_fn(examples):
    indices = [ex["index"] for ex in examples]
    texts = [formatting_prompts_func(example) for example in examples]
    tokenized = tokenizer(
        texts,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    tokenized["indices"] = indices
    return tokenized


# Create DataLoader using only the rows that need processing.
dataloader = DataLoader(
    to_process_df.to_dict(orient="records"),
    batch_size=batch_size,
    collate_fn=collate_fn
)

processed_rows = 0
for batch in tqdm(dataloader):
    input_ids = batch["input_ids"].to(model.device)
    attention_mask = batch.get("attention_mask", None)
    if attention_mask is not None:
        attention_mask = attention_mask.to(model.device)
    prompt_padded_len = len(input_ids[0])

    gen_tokens = model.generate(
        input_ids,
        attention_mask=attention_mask,
        temperature=parameters.baseline.temperature,
        top_p=parameters.baseline.top_p,
        top_k=parameters.baseline.top_k,
        max_new_tokens=max_new_tokens,
        do_sample=True,
    )

    # Remove the prompt tokens from the generated tokens
    gen_tokens = [gt[prompt_padded_len:] for gt in gen_tokens]

    # Decode generated tokens to text corrections
    corrections = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)
    corrections = ["".join(c) for c in corrections]

    # Update the original DataFrame using the indices provided in the batch.
    for idx, corr in zip(batch["indices"], corrections):
        test_df.loc[idx, "target"] = corr
        processed_rows += 1

        if processed_rows % save_each == 0:
            test_df.to_csv(out_path, index=False)
            print(f"Saved progress after processing {processed_rows} rows.")

# Final save after processing all batches.
test_df.to_csv(out_path, index=False)
print("Final save complete!")

In [ ]:
from google.colab import runtime
runtime.unassign()